In [1]:
from pylabnet.utils.logging.logger import LogClient
from pylabnet.utils.logging.logger import LogHandler
import time

This code does replicate the logger crashing. It does not seem to be connected with a memory issue, as the memory usage seems to be low.


![Alt text](image.png)

It does not look like the crashing can be mitigated by increaseing the waitime between two consecutive logging statements. Interestingly, if the logger crashed, it can be force-quit and restarted. After restarting, the log-messages appear (if the DoS code is still running). This is indicative with the filling up of the textbox being the issue. 


Let's log the number of logged chars before crashing occurs for different parameters:

| n_sleep      | num_sentence | n_lgs       | max_chars before crash    |
| -----------  | -----------  |-----------  |-----------  |
| 0.01       | 10000        |100        |18Mio        |
| 0.1     | 10000         | 100        | 15Mio        | 
| 0.5     | 10000         | 100        | 18Mio        | 
| 0.01     | 1000         | 10000        | 80Mio        | 


Some progress: After changing the window type of the "terminal" and "buffer_terminal" from `QTextBrowser` to `QPlainTextEdit` (which is optimized for plain text), we can spam the logger much more without crashing:

| n_sleep      | num_sentence | n_lgs       | crash yes no| number of chars     |
| -----------  | -----------  |-----------  |-----------  | ----------- |
| 0.01         | 100000       |100          | no          | 500Mio       |
| 0.01         | 10000000     |100          |  yes             |        |
| 0.01         | 1000000      |100          |  yes             |        |
| 0.01         | 1000000      |1000          |  yes             | after logfile was ~1GB --> 1000Mio chars      |

For log messages > 100000, we get this error: `terminate called after throwing an instance of 'std::bad_alloc'` --> We're actually running into a memory allocation problem. SHould be no problem though, since our single messages rarely will be > a couple 1000s of chars.


Maximium textsize in logger expanded from ~20Mio to 1000 Mio. 

Testing some other cases

| n_sleep      | num_sentence | n_lgs       | crash yes no| number of chars     |
| -----------  | -----------  |-----------  |-----------  | ----------- |
| 0.01         | 10       |100000          |     no but got impations      |  40Mio     |


In [2]:
# Instantiate
logger = LogClient(
    host='192.168.50.101',
    port=38967,
    module_tag='DoS Attack'
)


log = LogHandler(logger=logger)

In [6]:
num_sentence = 10
sentence = "I can definitly make you crash, let's find out how!" * num_sentence
end = "####################################" * 20

In [7]:
n_lgs = 100000
n_sleep = 0.01


In [8]:
len_sentence = len(sentence) + len(end)
char_counter = 0

In [9]:
for i in range(int(n_lgs)):

    char_log__msg = f"{char_counter} chars since start."

    time.sleep(n_sleep)
    log.info(sentence)
    time.sleep(n_sleep)
    log.info(end)
    time.sleep(n_sleep)
    log.info(char_log__msg)


    char_counter += len_sentence
    char_counter += len(char_log__msg)




KeyboardInterrupt: 

In [30]:
char_log__msg = f"{char_counter} chars since start."

time.sleep(n_sleep)
log.info(sentence)
time.sleep(n_sleep)
log.info(end)
time.sleep(n_sleep)
log.info(char_log__msg)


char_counter += len_sentence
char_counter += len(char_log__msg)